In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = (
    SparkSession.builder.appName("ExampleApp")
    .master("spark://spark-master:7077")  # Use the container hostname
    .config("spark.executor.memory", "1G")
    .config("spark.driver.memory", "1G")
    .getOrCreate()
)

In [3]:
import fastf1 as f1

In [7]:
session = f1.get_session(2021, 7, 'R')

req         WARNING 	DEFAULT CACHE ENABLED! (24.0 KB) /home/jovyan/.cache/fastf1


In [8]:
session.load()

core           INFO 	Loading data for French Grand Prix - Race [v3.6.1]
INFO:fastf1.fastf1.core:Loading data for French Grand Prix - Race [v3.6.1]
req            INFO 	No cached data found for session_info. Loading data...
INFO:fastf1.fastf1.req:No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
INFO:fastf1.api:Fetching session info data...
req            INFO 	Data has been written to cache!
INFO:fastf1.fastf1.req:Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
INFO:fastf1.fastf1.req:No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
INFO:fastf1.api:Fetching driver list...
req            INFO 	Data has been written to cache!
INFO:fastf1.fastf1.req:Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
INFO:fastf1.fastf1.req:No cached data found for session_status_dat

In [9]:
session.get_driver('33')

DriverNumber                                                         33
BroadcastName                                              M VERSTAPPEN
Abbreviation                                                        VER
DriverId                                                 max_verstappen
TeamName                                                Red Bull Racing
TeamColor                                                        0600EF
TeamId                                                         red_bull
FirstName                                                           Max
LastName                                                     Verstappen
FullName                                                 Max Verstappen
HeadshotUrl           https://www.formula1.com/content/dam/fom-websi...
CountryCode                                                            
Position                                                            1.0
ClassifiedPosition                                              

In [10]:
circuit_info = session.get_circuit_info()

In [11]:
session.get_driver('44')

DriverNumber                                                         44
BroadcastName                                                L HAMILTON
Abbreviation                                                        HAM
DriverId                                                       hamilton
TeamName                                                       Mercedes
TeamColor                                                        00D2BE
TeamId                                                         mercedes
FirstName                                                         Lewis
LastName                                                       Hamilton
FullName                                                 Lewis Hamilton
HeadshotUrl           https://www.formula1.com/content/dam/fom-websi...
CountryCode                                                            
Position                                                            2.0
ClassifiedPosition                                              

In [12]:
verstapan = session.laps.pick_driver('33').pick_lap(1)

/opt/conda/lib/python3.11/site-packages/fastf1/core.py:3183: FutureWarning: pick_driver is deprecated and will be removed in a future release. Use pick_drivers instead.
  warnings.warn(("pick_driver is deprecated and will be removed"
/opt/conda/lib/python3.11/site-packages/fastf1/core.py:3137: FutureWarning: pick_lap is deprecated and will be removed in a future release. Use pick_laps instead.
  warnings.warn(("pick_lap is deprecated and will be removed in a "


In [13]:
df = verstapan.get_car_data()

In [14]:
df.columns

Index(['Date', 'RPM', 'Speed', 'nGear', 'Throttle', 'Brake', 'DRS', 'Source',
       'Time', 'SessionTime'],
      dtype='object')

In [15]:
spark_df = spark.createDataFrame(data=df, schema=list(df.columns))

In [16]:
spark_df.show(25)

+--------------------+-------+-----+-----+--------+-----+---+------+--------------------+--------------------+
|                Date|    RPM|Speed|nGear|Throttle|Brake|DRS|Source|                Time|         SessionTime|
+--------------------+-------+-----+-----+--------+-----+---+------+--------------------+--------------------+
|2021-06-20 13:03:...|11078.0|  0.0|    1|    16.0| true|  1|   car|INTERVAL '0 00:00...|INTERVAL '0 00:33...|
|2021-06-20 13:03:...|10644.0|  0.0|    1|    16.0|false|  1|   car|INTERVAL '0 00:00...|INTERVAL '0 00:33...|
|2021-06-20 13:03:...| 8964.0|  8.0|    1|    16.0|false|  1|   car|INTERVAL '0 00:00...|INTERVAL '0 00:33...|
|2021-06-20 13:03:...| 7284.0| 15.0|    1|    17.0|false|  1|   car|INTERVAL '0 00:00...|INTERVAL '0 00:33...|
|2021-06-20 13:03:...| 5604.0| 29.0|    1|    20.0|false|  1|   car|INTERVAL '0 00:00...|INTERVAL '0 00:33...|
|2021-06-20 13:03:...| 4446.0| 32.0|    1|    21.0|false|  1|   car|INTERVAL '0 00:00...|INTERVAL '0 00:33...|
|

In [27]:
session.get

ModuleNotFoundError: No module named 'pyspark.sql.Functions'

In [24]:
spark_df.write.option('header', True).option('delimiter', ',').csv('test.csv')


Py4JJavaError: An error occurred while calling o71.csv.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 4.0 failed 4 times, most recent failure: Lost task 0.3 in stage 4.0 (TID 186) (172.19.0.4 executor 0): java.io.IOException: Mkdirs failed to create file:/home/jovyan/work/test.csv/_temporary/0/_temporary/attempt_202509102037489000607268166805710_0004_m_000000_186 (exists=false, cwd=file:/opt/bitnami/spark/work/app-20250910202603-0002/0)
	at org.apache.hadoop.fs.ChecksumFileSystem.create(ChecksumFileSystem.java:515)
	at org.apache.hadoop.fs.ChecksumFileSystem.create(ChecksumFileSystem.java:500)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1195)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1175)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1064)
	at org.apache.spark.sql.execution.datasources.CodecStreams$.createOutputStream(CodecStreams.scala:81)
	at org.apache.spark.sql.execution.datasources.CodecStreams$.c